In [2]:
!nvidia-smi

Thu May 20 11:38:40 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    24W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
import os
# your workspace in your drive
workspace = 'ML2021-hw7'

try:
  os.chdir(os.path.join('/content/gdrive/MyDrive/', workspace))
except:
  os.makedirs(os.path.join('/content/gdrive/MyDrive/', workspace), exist_ok=True)
  os.chdir(os.path.join('/content/gdrive/MyDrive/', workspace))

In [5]:
# You are allowed to change version of transformers or use other toolkits
!pip install transformers==4.5.0

     |████████████████████████████████| 2.2MB 4.0MB/s 
     |████████████████████████████████| 3.3MB 37.7MB/s 
     |████████████████████████████████| 901kB 47.6MB/s 


In [6]:
import json
import numpy as np
import random
import torch
from torch.utils.data import DataLoader, Dataset 
from transformers import AdamW, BertForQuestionAnswering, BertTokenizer, get_linear_schedule_with_warmup, BertModel, BertTokenizerFast

from tqdm.auto import tqdm

device = "cuda" if torch.cuda.is_available() else "cpu"

# Fix random seed for reproducibility
def same_seeds(seed):
	  torch.manual_seed(seed)
	  if torch.cuda.is_available():
		    torch.cuda.manual_seed(seed)
		    torch.cuda.manual_seed_all(seed)
	  np.random.seed(seed)
	  random.seed(seed)
	  torch.backends.cudnn.benchmark = False
	  torch.backends.cudnn.deterministic = True
same_seeds(0)

In [7]:
# Change "fp16_training" to True to support automatic mixed precision training (fp16)	
fp16_training = True

if fp16_training:
    !pip install accelerate==0.2.0
    from accelerate import Accelerator
    accelerator = Accelerator(fp16=True)
    device = accelerator.device

# Documentation for the toolkit:  https://huggingface.co/docs/accelerate/

     |████████████████████████████████| 51kB 2.9MB/s 


In [8]:
tokenizer = BertTokenizerFast.from_pretrained("hfl/chinese-macbert-large")
model = BertForQuestionAnswering.from_pretrained("hfl/chinese-macbert-large").to(device)

# You can safely ignore the warning message (it pops up because new prediction heads for QA are initialized randomly)

Some weights of the model checkpoint at hfl/chinese-macbert-large were not used when initializing BertForQuestionAnswering: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the

In [9]:
def read_data(file):
    with open(file, 'r', encoding="utf-8") as reader:
        data = json.load(reader)
    return data["questions"], data["paragraphs"]

train_questions, train_paragraphs = read_data("hw7_train.json")
dev_questions, dev_paragraphs = read_data("hw7_dev.json")
test_questions, test_paragraphs = read_data("hw7_test.json")

In [10]:
# Tokenize questions and paragraphs separately
# 「add_special_tokens」 is set to False since special tokens will be added when tokenized questions and paragraphs are combined in datset __getitem__ 

train_questions_tokenized = tokenizer([train_question["question_text"] for train_question in train_questions], add_special_tokens=False)
dev_questions_tokenized = tokenizer([dev_question["question_text"] for dev_question in dev_questions], add_special_tokens=False)
test_questions_tokenized = tokenizer([test_question["question_text"] for test_question in test_questions], add_special_tokens=False) 

train_paragraphs_tokenized = tokenizer(train_paragraphs, add_special_tokens=False)
dev_paragraphs_tokenized = tokenizer(dev_paragraphs, add_special_tokens=False)
test_paragraphs_tokenized = tokenizer(test_paragraphs, add_special_tokens=False)

# You can safely ignore the warning message as tokenized sequences will be futher processed in datset __getitem__ before passing to model

In [11]:
class QA_Dataset(Dataset):
    def __init__(self, split, questions, tokenized_questions, tokenized_paragraphs):
        self.split = split
        self.questions = questions
        self.tokenized_questions = tokenized_questions
        self.tokenized_paragraphs = tokenized_paragraphs
        self.max_question_len = 40
        self.max_paragraph_len = 350
        
        ##### TODO: Change value of doc_stride #####
        self.doc_stride = 300

        # Input sequence length = [CLS] + question + [SEP] + paragraph + [SEP]
        self.max_seq_len = 1 + self.max_question_len + 1 + self.max_paragraph_len + 1

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):
        question = self.questions[idx]
        tokenized_question = self.tokenized_questions[idx]
        tokenized_paragraph = self.tokenized_paragraphs[question["paragraph_id"]]

        ##### TODO: Preprocessing #####
        # Hint: How to prevent model from learning something it should not learn

        if self.split == "train":
            # Convert answer's start/end positions in paragraph_text to start/end positions in tokenized_paragraph  
            answer_start_token = tokenized_paragraph.char_to_token(question["answer_start"])
            answer_end_token = tokenized_paragraph.char_to_token(question["answer_end"])

            # A single window is obtained by slicing the portion of paragraph containing the answer
            mid = int((answer_start_token + answer_end_token) // (2+random.uniform(-1,1)))
            paragraph_start = max(0, min(mid - self.max_paragraph_len // 2, len(tokenized_paragraph) - self.max_paragraph_len))
            paragraph_end = paragraph_start + self.max_paragraph_len

            # Slice question/paragraph and add special tokens (101: CLS, 102: SEP)
            input_ids_question = [101] + tokenized_question.ids[:self.max_question_len] + [102] 
            input_ids_paragraph = tokenized_paragraph.ids[paragraph_start : paragraph_end] + [102]		
            
            # Convert answer's start/end positions in tokenized_paragraph to start/end positions in the window  
            answer_start_token += len(input_ids_question) - paragraph_start
            answer_end_token += len(input_ids_question) - paragraph_start
            
            # Pad sequence and obtain inputs to model 
            input_ids, token_type_ids, attention_mask = self.padding(input_ids_question, input_ids_paragraph)
            return torch.tensor(input_ids), torch.tensor(token_type_ids), torch.tensor(attention_mask), answer_start_token, answer_end_token

        # Validation/Testing
        else:
            input_ids_list, token_type_ids_list, attention_mask_list = [], [], []
            
            # Paragraph is split into several windows, each with start positions separated by step "doc_stride"
            for i in range(0, len(tokenized_paragraph), self.doc_stride):
                
                # Slice question/paragraph and add special tokens (101: CLS, 102: SEP)
                input_ids_question = [101] + tokenized_question.ids[:self.max_question_len] + [102]
                input_ids_paragraph = tokenized_paragraph.ids[i : i + self.max_paragraph_len] + [102]
                
                # Pad sequence and obtain inputs to model
                input_ids, token_type_ids, attention_mask = self.padding(input_ids_question, input_ids_paragraph)
                
                input_ids_list.append(input_ids)
                token_type_ids_list.append(token_type_ids)
                attention_mask_list.append(attention_mask)
            
            return torch.tensor(input_ids_list), torch.tensor(token_type_ids_list), torch.tensor(attention_mask_list)

    def padding(self, input_ids_question, input_ids_paragraph):
        # Pad zeros if sequence length is shorter than max_seq_len
        padding_len = self.max_seq_len - len(input_ids_question) - len(input_ids_paragraph)
        # Indices of input sequence tokens in the vocabulary
        input_ids = input_ids_question + input_ids_paragraph + [0] * padding_len
        # Segment token indices to indicate first and second portions of the inputs. Indices are selected in [0, 1]
        token_type_ids = [0] * len(input_ids_question) + [1] * len(input_ids_paragraph) + [0] * padding_len
        # Mask to avoid performing attention on padding token indices. Mask values selected in [0, 1]
        attention_mask = [1] * (len(input_ids_question) + len(input_ids_paragraph)) + [0] * padding_len
        
        return input_ids, token_type_ids, attention_mask

train_set = QA_Dataset("train", train_questions, train_questions_tokenized, train_paragraphs_tokenized)
dev_set = QA_Dataset("dev", dev_questions, dev_questions_tokenized, dev_paragraphs_tokenized)
test_set = QA_Dataset("test", test_questions, test_questions_tokenized, test_paragraphs_tokenized)

train_batch_size = 8

# Note: Do NOT change batch size of dev_loader / test_loader !
# Although batch size=1, it is actually a batch consisting of several windows from the same QA pair
train_loader = DataLoader(train_set, batch_size=train_batch_size, shuffle=True, pin_memory=True)
dev_loader = DataLoader(dev_set, batch_size=1, shuffle=False, pin_memory=True)
test_loader = DataLoader(test_set, batch_size=1, shuffle=False, pin_memory=True)

In [12]:
def index_before_tokenize(tokens, start, end):
    char_count, new_start, new_end = 0, 512, 512
    start_flag = 0
    end_flag = 0
        
    for i, token in enumerate(tokens):
        if token == '[UNK]' or token == '[CLS]' or token == '[SEP]':
            if i == start:
                new_start = char_count
            if i == end:
                new_end = char_count
            char_count += 1
        else:
            for c in token:
                if i == start and start_flag == 0:
                    #print(token)
                    new_start = char_count
                    start_flag = 1
                if i == end:
                    #print(token)
                    new_end = char_count
                    end_flag = 1
                if c != '#':
                    char_count += 1
    return new_start, new_end

In [13]:
def evaluate_fusion(data, output1, output2, output3, doc_stride=150, paragraph=None, paragraph_tokenized=None):
    ##### TODO: Postprocessing #####
    # There is a bug and room for improvement in postprocessing 
    # Hint: Open your prediction file to see what is wrong 
    
    answer = ''
    max_prob = float('-inf')
    num_of_windows = data[0].shape[1]
    
    # index in the whole tokens (not just relative to window)
    entire_start_index = 0
    entire_end_index = 0
    start_logits = (output1.start_logits + output2.start_logits + output3.start_logits) / 3
    end_logits = (output1.end_logits + output2.end_logits + output3.end_logits) / 3
    
    for k in range(num_of_windows):
        #print('window',k)
        # Obtain answer by choosing the most probable start position / end position
        mask = data[1][0][k].bool() &  data[2][0][k].bool() # token type & attention mask
        masked_output_start = torch.masked_select(start_logits[k], mask)[:-1] # -1 is [SEP]
        start_prob, start_index = torch.max(masked_output_start, dim=0)
        masked_output_end = torch.masked_select(end_logits[k], mask)[start_index:-1] # -1 is [SEP]
        end_prob, end_index = torch.max(masked_output_end, dim=0)
        end_index += start_index 
        

        # Probability of answer is calculated as sum of start_prob and end_prob
        prob = start_prob + end_prob
        masked_data = torch.masked_select(data[0][0][k], mask)[:-1] # -1 is [SEP]

        # Replace answer if calculated probability is larger than previous windows
        if (prob > max_prob) and (end_index >= start_index) and (end_index - start_index <= 30):
            max_prob = prob
            entire_start_index = start_index.item() + doc_stride * k
            entire_end_index = end_index.item() + doc_stride * k
            #print('entire_start_index',entire_start_index)
            #print('entire_end_index',entire_end_index)
            # Convert tokens to chars (e.g. [1920, 7032] --> "大 金")
            answer = tokenizer.decode(masked_data[start_index : end_index + 1])
            # Remove spaces in answer (e.g. "大 金" --> "大金")
            answer = answer.replace('✔', ' ').replace('✦','\u200b').replace('☺','\u200e').replace('☆','\u3000').replace('●','#').replace(' ','')

    
    # if [UNK] in prediction, use orignal span of paragrah
    if '[UNK]' in answer:
        print('found [UNK] in prediction, using original text')
        print('original prediction', answer)
        # find the index of answer in the orinal paragrah

        new_start, new_end = index_before_tokenize(tokens=paragraph_tokenized, 
                                                   start=entire_start_index, end=entire_end_index)
        #print('new_start',new_start)
        #print('new_end',new_end)
        answer = paragraph[new_start:new_end+1]
        answer = answer.replace('✔', ' ').replace('✦','\u200b').replace('☺','\u200e').replace('☆','\u3000').replace('●','#')
        print('final prediction',answer)

            
    
    return answer

In [14]:
model1 = BertForQuestionAnswering.from_pretrained("saved_model/macbert4").to(device)
model2 = BertForQuestionAnswering.from_pretrained("saved_model/macbert4_val").to(device)
model3 = BertForQuestionAnswering.from_pretrained("saved_model/macbert6").to(device)

In [15]:
print("Evaluating Test Set ...")

result = []

model1.eval()
model2.eval()
model3.eval()
with torch.no_grad():
    for i, data in enumerate(tqdm(test_loader)):
        output1 = model1(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
                       attention_mask=data[2].squeeze(dim=0).to(device))
        output2 = model2(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
                       attention_mask=data[2].squeeze(dim=0).to(device))
        output3 = model3(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
                       attention_mask=data[2].squeeze(dim=0).to(device))

        result.append(evaluate_fusion(data, output1, output2, output3, 300, test_paragraphs[test_questions[i]['paragraph_id']],
                               test_paragraphs_tokenized[test_questions[i]['paragraph_id']].tokens))

result_file = "/content/gdrive/MyDrive/result_fusion.csv"
with open(result_file, 'w') as f:	
	  f.write("ID,Answer\n")
	  for i, test_question in enumerate(test_questions):
        # Replace commas in answers with empty strings (since csv is separated by comma)
        # Answers in kaggle are processed in the same way
		    f.write(f"{test_question['id']},{result[i].replace(',','')}\n")

print(f"Completed! Result is in {result_file}")

Evaluating Test Set ...


found [UNK] in prediction, using original text
original prediction 溥[UNK]
final prediction 溥儁
found [UNK] in prediction, using original text
original prediction 目前沒有觀察到任何語言純[UNK]以力道來區分不同輔音
final prediction 目前沒有觀察到任何語言純綷以力道來區分不同輔音
found [UNK] in prediction, using original text
original prediction [UNK]人國
final prediction 荇人國
found [UNK] in prediction, using original text
original prediction [UNK]
final prediction 株
found [UNK] in prediction, using original text
original prediction 馬[UNK]
final prediction 馬馼
found [UNK] in prediction, using original text
original prediction 東晉常[UNK]
final prediction ，東晉常
found [UNK] in prediction, using original text
original prediction [UNK]稻
final prediction 秈稻
found [UNK] in prediction, using original text
original prediction 白[UNK]紀滅絕事件
final prediction 白堊紀滅絕事件
found [UNK] in prediction, using original text
original prediction 抗佝[UNK]病維他命
final prediction 抗佝僂病維他命
found [UNK] in prediction, using original text
original prediction 杭州[UNK]橋機場
final pred

## Kaggle puclic: 0.85926